<a href="https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/Donut/CORD/Fine_tune_Donut_on_a_custom_dataset_(CORD)_with_PyTorch_Lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I chose [this notebook](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/Donut/CORD/Fine_tune_Donut_on_a_custom_dataset_(CORD)_with_PyTorch_Lightning.ipynb) as a baseline for my work. Here I will finetune pre-trained CORD model on a synthetic [dataset](https://github.com/GitYCC/traditional-chinese-text-recogn-dataset).  

We begin with installation (I would recommend running this notebook in colab):

In [ ]:
!pip install -q transformers datasets sentencepiece

In [2]:
!pip install -q pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 14.6 MB/s eta 0:00:00


I chose raw dataset, simple preprocessing happening here:

In [3]:
!tar -xvf /content/TCSynth-VAL_raw.tar

Выходные данные были обрезаны до нескольких последних строк (5000).
TCSynth-VAL_raw/0000/00001001.jpg
TCSynth-VAL_raw/0000/00001002.jpg
TCSynth-VAL_raw/0000/00001004.jpg
TCSynth-VAL_raw/0000/00001003.jpg
TCSynth-VAL_raw/0000/00001005.jpg
TCSynth-VAL_raw/0000/00001006.jpg
TCSynth-VAL_raw/0000/00001007.jpg
TCSynth-VAL_raw/0000/00001008.jpg
TCSynth-VAL_raw/0000/00001009.jpg
TCSynth-VAL_raw/0000/00001010.jpg
TCSynth-VAL_raw/0000/00001011.jpg
TCSynth-VAL_raw/0000/00001012.jpg
TCSynth-VAL_raw/0000/00001013.jpg
TCSynth-VAL_raw/0000/00001014.jpg
TCSynth-VAL_raw/0000/00001015.jpg
TCSynth-VAL_raw/0000/00001016.jpg
TCSynth-VAL_raw/0000/00001017.jpg
TCSynth-VAL_raw/0000/00001018.jpg
TCSynth-VAL_raw/0000/00001019.jpg
TCSynth-VAL_raw/0000/00001020.jpg
TCSynth-VAL_raw/0000/00001021.jpg
TCSynth-VAL_raw/0000/00001022.jpg
TCSynth-VAL_raw/0000/00001023.jpg
TCSynth-VAL_raw/0000/00001024.jpg
TCSynth-VAL_raw/0000/00001025.jpg
TCSynth-VAL_raw/0000/00001026.jpg
TCSynth-VAL_raw/0000/00001027.jpg
TCSynth-VAL_ra

In [4]:
from PIL import Image

data = {'image': [], 'text':[]}
parent_dir = '/content/TCSynth-VAL_raw/'
file_name = 'labels.txt'

with open(parent_dir + file_name) as f:
    for line in f:
        path, text = line.split()
        image = Image.open(parent_dir + path).convert('RGB')
        data['image'].append(image)
        data['text'].append(text)

In [5]:
from datasets import Dataset

full_chinese = Dataset.from_dict(data).train_test_split(test_size=0.1)


## Dataset


In [6]:
full_chinese

DatasetDict({
    train: Dataset({
        features: ['image', 'text'],
        num_rows: 5400
    })
    test: Dataset({
        features: ['image', 'text'],
        num_rows: 600
    })
})

Chinese examples look like this:

In [7]:
example = full_chinese['train'][0]
image = example['image']
width, height = image.size
display(image)

## Load model and processor

Here we get pre-trained weights for Donut model for similar task, CORD (there are extra tokens but we will get rid of them on the inference stage).

In [43]:
from transformers import VisionEncoderDecoderConfig

image_size = [1280, 960]
max_length = 60

config = VisionEncoderDecoderConfig.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")
config.encoder.image_size = image_size
config.decoder.max_length = max_length

Next, we instantiate the model with our custom config, as well as the processor. 

In [44]:
from transformers import DonutProcessor, VisionEncoderDecoderModel

processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2", config=config)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


## Create PyTorch dataset

Basic dataset on top of our data:


In [45]:
from typing import Tuple

import torch
from torch.utils.data import Dataset

added_tokens = []

class DonutDataset(Dataset):
    """
    PyTorch Dataset for Donut. This class takes a HuggingFace Dataset as input.

    Each row, consists of image path(png/jpg/jpeg) and gt data (json/jsonl/txt),
    and it will be converted into pixel_values (vectorized image) and labels (input_ids of the tokenized string).

    Args:
        dataset_name_or_path: name of dataset (available at huggingface.co/datasets) or the path containing image files and metadata.jsonl
        max_length: the max number of tokens for the target sequences
        split: whether to load "train", "validation" or "test" split
        ignore_id: ignore_index for torch.nn.CrossEntropyLoss
    """

    def __init__(
        self,
        dataset: Dataset,
        max_length: int,
        split: str = "train",
        ignore_id: int = -100,
    ):
        super().__init__()

        self.max_length = max_length
        self.split = split
        self.ignore_id = ignore_id

        self.dataset = dataset
        self.dataset_length = len(self.dataset)
        self.ground_truth = []

        for sample in self.dataset:
            self.ground_truth.append(sample['text'] + processor.tokenizer.eos_token)

    def __len__(self) -> int:
        return self.dataset_length

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Load image from image_path of given dataset_path and convert into input_tensor and labels
        Convert gt data into input_ids (tokenized string)
        Returns:
            input_tensor : preprocessed image
            input_ids : tokenized gt_data
            labels : masked labels (model doesn't need to predict prompt and pad token)
        """
        sample = self.dataset[idx]


        # inputs
        pixel_values = processor(sample["image"], random_padding=self.split == "train", return_tensors="pt").pixel_values
        pixel_values = pixel_values.squeeze()

        # targets
        target_sequence = self.ground_truth[idx]
        input_ids = processor.tokenizer(
            target_sequence,
            add_special_tokens=False,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )["input_ids"].squeeze(0)

        labels = input_ids.clone()
        labels[labels == processor.tokenizer.pad_token_id] = self.ignore_id  # model doesn't need to predict pad token
        return pixel_values, labels, target_sequence

Next, we instantiate the datasets:

In [46]:
processor.image_processor.size = image_size[::-1]
processor.image_processor.do_align_long_axis = False

train_dataset = DonutDataset(full_chinese['train'], max_length=max_length,
                             split="train"
                             )

val_dataset = DonutDataset(full_chinese['test'], max_length=max_length,
                             split="test"
                             )

In [47]:
pixel_values, labels, target_sequence = train_dataset[0]

In [48]:
print(pixel_values.shape)

torch.Size([3, 1280, 960])


And this is how the labels looks like:

In [49]:

for id in labels.tolist()[:30]:
  if id != -100:
    print(processor.decode([id]))
  else:
    print(id)


保
華
集團
有限公司
</s>
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100


In [50]:
print(target_sequence)

保華集團有限公司</s>


In [51]:
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.decoder_start_token_id = processor.tokenizer.convert_tokens_to_ids(['<s_cord-v2>'])[0]

In [52]:
# sanity check
print("Pad token ID:", processor.decode([model.config.pad_token_id]))
print("Decoder start token ID:", processor.decode([model.config.decoder_start_token_id]))

Pad token ID: <pad>
Decoder start token ID: <s_cord-v2>


## Create PyTorch DataLoaders

The choice of the batch isze is limited due to Colab memory restrictions

In [53]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=4)

Let's verify a batch:

In [54]:
batch = next(iter(train_dataloader))
pixel_values, labels, target_sequences = batch
print(pixel_values.shape)

torch.Size([1, 3, 1280, 960])


In [55]:
for id in labels.squeeze().tolist()[:30]:
  if id != -100:
    print(processor.decode([id]))
  else:
    print(id)


加
值
資料
庫
</s>
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100
-100


In [56]:
print(len(train_dataset))
print(len(val_dataset))

5400
600


In [57]:
batch = next(iter(val_dataloader))
pixel_values, labels, target_sequences = batch
print(pixel_values.shape)

torch.Size([1, 3, 1280, 960])


In [58]:
print(target_sequences[0])

日喀則和平機場</s>


In [110]:
import re
from nltk import edit_distance
import numpy as np

import pytorch_lightning as pl


class DonutModelPLModule(pl.LightningModule):
    def __init__(self, config, processor, model):
        super().__init__()
        self.config = config
        self.processor = processor
        self.model = model

    def training_step(self, batch, batch_idx):
        pixel_values, labels, _ = batch

        outputs = self.model(pixel_values, labels=labels)
        loss = outputs.loss
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx, dataset_idx=0):
        pixel_values, labels, answers = batch
        batch_size = pixel_values.shape[0]
        print(pixel_values.shape)
        decoder_input_ids = torch.full((batch_size, 1), self.model.config.decoder_start_token_id, device=self.device)

        outputs = self.model.generate(pixel_values,
                                   decoder_input_ids=decoder_input_ids,
                                   max_length=max_length,
                                   early_stopping=True,
                                   pad_token_id=self.processor.tokenizer.pad_token_id,
                                   eos_token_id=self.processor.tokenizer.eos_token_id,
                                   use_cache=True,
                                   num_beams=1,
                                   bad_words_ids=[[self.processor.tokenizer.unk_token_id]],
                                   return_dict_in_generate=True,)

        predictions = []
        for seq in self.processor.tokenizer.batch_decode(outputs.sequences):
            seq = seq.replace(self.processor.tokenizer.eos_token, "").replace(self.processor.tokenizer.pad_token, "")
            seq = re.sub(r"<.*?>", "", seq, count=1).strip()  # remove all tokens if there are any
            predictions.append(seq)

        scores = []
        for pred, answer in zip(predictions, answers):
            pred = re.sub(r"(?:(?<=>) | (?=</s_))", "", pred)
            answer = answer.replace(self.processor.tokenizer.eos_token, "")
            scores.append(edit_distance(pred, answer) / max(len(pred), len(answer)))

            if self.config.get("verbose", False) and len(scores) == 1:
                print(f"Prediction: {pred}")
                print(f"    Answer: {answer}")
                print(f" Normed ED: {scores[0]}")

        self.log("val_edit_distance", np.mean(scores))

        return scores

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.config.get("lr"))

        return optimizer

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return val_dataloader

## Training

Training parameters:

In [112]:
config = {"max_epochs":1,
          "val_check_interval":0.001,
          "check_val_every_n_epoch":1,
          "gradient_clip_val":1.0,
          "num_training_samples_per_epoch": 800,
          "lr":3e-5,
          "train_batch_sizes": [8],
          "val_batch_sizes": [1],
          "num_nodes": 1,
          "warmup_steps": 300,
          "result_path": "./result",
          "verbose": True,
          }

model_module = DonutModelPLModule(config, processor, model)

In [113]:
from pytorch_lightning.callbacks import EarlyStopping

early_stop_callback = EarlyStopping(monitor="val_edit_distance", patience=3, verbose=False, mode="min")

trainer = pl.Trainer(
        accelerator="gpu",
        devices=1,
        max_epochs=config.get("max_epochs"),
        val_check_interval=config.get("val_check_interval"),
        check_val_every_n_epoch=config.get("check_val_every_n_epoch"),
        gradient_clip_val=config.get("gradient_clip_val"),
        precision=16,
        num_sanity_val_steps=0,
)

trainer.fit(model_module)

/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:558: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                      | Params
----------------------------------------------------
0 | model | VisionEncoderDecoderModel | 201 M 
----------------------------------------------------
201 M     Trainable params
0         Non-trainable para

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

torch.Size([1, 3, 1280, 960])


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Prediction: 日略則和平機場
    Answer: 日喀則和平機場
 Normed ED: 0.14285714285714285
torch.Size([1, 3, 1280, 960])
Prediction: 花子
    Answer: 耙子
 Normed ED: 0.5
torch.Size([1, 3, 1280, 960])
Prediction: 旋回棋
    Answer: 旋回棋
 Normed ED: 0.0
torch.Size([1, 3, 1280, 960])
Prediction: 裝甲兵技術學校
    Answer: 裝甲兵技術學校
 Normed ED: 0.0
torch.Size([1, 3, 1280, 960])
Prediction: 中學軍隊
    Answer: 劉吳惠蘭
 Normed ED: 1.0
torch.Size([1, 3, 1280, 960])
Prediction: 多爾人大橋
    Answer: 卡爾代朗山脈
 Normed ED: 0.8333333333333334
torch.Size([1, 3, 1280, 960])
Prediction: 美國退伍軍人事務部長
    Answer: 美國退伍軍人事務部長
 Normed ED: 0.0
torch.Size([1, 3, 1280, 960])
Prediction: 西偏偏偏偏列
    Answer: 西瑞斯河
 Normed ED: 0.8333333333333334
torch.Size([1, 3, 1280, 960])
Prediction: 飛泊
    Answer: 盤道
 Normed ED: 1.0
torch.Size([1, 3, 1280, 960])
Prediction: 水庫
    Answer: 金坑文昌宮
 Normed ED: 1.0
torch.Size([1, 3, 1280, 960])
Prediction: 康第1/丑斯
    Answer: 康第紐斯
 Normed ED: 0.5
torch.Size([1, 3, 1280, 960])
Prediction: 孟梅拉國
    Answer: 孟加拉國共產黨
 Normed ED: 0.571

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


## Final steps

Weights are stored in ckpt file, we can load others from colab if needed:

In [77]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [96]:
model2 = getattr(DonutModelPLModule.load_from_checkpoint('/content/drive/MyDrive/TZ GEN/epoch=0-step=5400.ckpt', config= config, model=model, processor=processor), 'model')

In [133]:
task_prompt = ""


device = "cuda" if torch.cuda.is_available() else "cpu"
decoder_input_ids = torch.full((1, 1), model2.config.decoder_start_token_id, device=device)
model2.to(device)
pix_vals = processor(image, return_tensors="pt").pixel_values

for batch in val_dataset:
    pixel_values, labels, answer = batch
    batch_size = pixel_values.shape[0]
    outputs = model2.generate(pixel_values.unsqueeze(0).to(device),
                               decoder_input_ids=decoder_input_ids.to(device),
                               max_length=model.decoder.config.max_position_embeddings,
                               early_stopping=True,
                               pad_token_id=processor.tokenizer.pad_token_id,
                               eos_token_id=processor.tokenizer.eos_token_id,
                               use_cache=True,
                               num_beams=1,
                               bad_words_ids=[[processor.tokenizer.unk_token_id]],
                               return_dict_in_generate=True,
                               output_scores=True,)

    predictions = []
    for seq in processor.tokenizer.batch_decode(outputs.sequences):
        seq = seq.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
        seq = re.sub(r"<.*?>", "", seq, count=1).strip()  # remove first task start token
        predictions.append(seq)

    scores = []

    pred = re.sub(r"(?:(?<=>) | (?=</s_))", "", predictions[0])
    answer = answer.replace(processor.tokenizer.eos_token, "")
    scores.append(edit_distance(pred, answer) / max(len(pred), len(answer)))

    if config.get("verbose", False) and len(scores) == 1:
        print(f"Prediction: {pred}")
        print(f"    Answer: {answer}")
        print(f" Normed ED: {scores[0]}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Prediction: 日略則和平機場
    Answer: 日喀則和平機場
 Normed ED: 0.14285714285714285
Prediction: 花子
    Answer: 耙子
 Normed ED: 0.5
Prediction: 旋回棋
    Answer: 旋回棋
 Normed ED: 0.0
Prediction: 裝甲兵技術學校
    Answer: 裝甲兵技術學校
 Normed ED: 0.0
Prediction: 中學軍樓
    Answer: 劉吳惠蘭
 Normed ED: 1.0
Prediction: 多爾人大橋
    Answer: 卡爾代朗山脈
 Normed ED: 0.8333333333333334
Prediction: 美國退伍軍人事務部長
    Answer: 美國退伍軍人事務部長
 Normed ED: 0.0
Prediction: 西偏偏偏偏列
    Answer: 西瑞斯河
 Normed ED: 0.8333333333333334
Prediction: 飛泊
    Answer: 盤道
 Normed ED: 1.0
Prediction: 水庫
    Answer: 金坑文昌宮
 Normed ED: 1.0
Prediction: 康第1/丑斯
    Answer: 康第紐斯
 Normed ED: 0.5
Prediction: 孟梅拉國
    Answer: 孟加拉國共產黨
 Normed ED: 0.5714285714285714
Prediction: 埃埃拉斯塔爾
    Answer: 坎波埃利亞斯市
 Normed ED: 0.8571428571428571
Prediction: 中核核
    Answer: 和誘罪
 Normed ED: 1.0
Prediction: 城后者廣
    Answer: 歧舌苔屬
 Normed ED: 1.0
Prediction: 紙倒房
    Answer: 黃鸝鳥
 Normed ED: 1.0
Prediction: 現排與教學
    Answer: 建構與教學
 Normed ED: 0.4
Prediction: 埃拉伊拉姆帕索
    Answer: 埃拉伊拉姆帕奈
 Normed 

ValueError: ignored